In [ ]:
# from ipywidgets import *
# from IPython.core.display import display



from ipywidgets import *
from ipywebrtc import CameraStream, ImageRecorder
from IPython.display import IFrame, YouTubeVideo
from IPython.core.display import display
from ipywidgets import HTML
import subprocess, sys
import base64

import cv2
import io
import numpy as np
import PIL.Image
from PIL import *

In [ ]:
gamma = 1
cl = 0
fo = 0

In [ ]:
####  Inversion  #####
def invert(_):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    #########################################
    iminvers = cv2.bitwise_not(imm)
    ####################################
    fmt = 'png'
    data_comp = compress_to_bytes(iminvers, fmt)
    Img.value=data_comp

In [ ]:
#################################
def Canny_transfo(_):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    #########################################"

    # Setting parameter values
    t_lower = 50  # Lower Threshold
    t_upper = 150  # Upper threshold

    # Applying the Canny Edge filter
    edge = cv2.Canny(imm, t_lower, t_upper)
    ####################################
    fmt = 'png'
    data_comp = compress_to_bytes(edge, fmt)
    Img.value=data_comp

In [ ]:
#################################
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########

In [ ]:
#############################################
def adjust_brightness(img, brightness_factor):
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################

In [ ]:
def Travail_img(gamma):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    imm=adjust_brightness(imm, gamma)
    #########################################"
    fmt = 'png'
    data_comp = compress_to_bytes(imm, fmt)
    Img.value=data_comp
    return gamma
    #####

In [ ]:
#########################               
def Contrast(_):
    global gamma
    cl=clair_z.value
    fo=sombre_z.value
    cl_new=cl_new_z.value
    fo_new=fo_new_z.value
    if cl>cl_new:
        if fo==0:
            cl_new+=1
            cl_new_z.value=cl_new #
        else:
            clair_z.value=cl_new
            fo_new=fo_new-1
            fo_new_z.value=fo_new
            sombre_z.value=fo_new

    if fo>fo_new:
        if cl==0:
            fo_new+=1
            fo_new_z.value=fo_new
        else:
            sombre_z.value=fo_new
            cl_new=cl_new-1
            cl_new_z.value=cl_new
            clair_z.value=cl_new

    if fo==0:
        gamma=float(cl/10)+1
    if cl==0:
        gamma=1-float(fo/10)
    Travail_img(gamma)
#########################

In [ ]:
######  Recup Image d'1 Vidéo  ########
def Recup_image(_):
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)#[..., :3]
    ####
#     mask_circle = np.zeros(im_array.shape[:2],np.uint8)
#     cv2.circle(mask_circle,(320,240), 50, 255, -1) # a.value, 255, -1)
#     img_rgb = cv2.bitwise_and(im_array, im_array, mask=mask_circle)    
    ####
    fmt = 'png'
    data_comp = compress_to_bytes(im_array, fmt)
#     image_headline.value = data_comp

    Img.value = data_comp

    ####
#     data_comp2 = compress_to_bytes(im_array, fmt)
#     image_headline2.value = data_comp2
    yo.layout.visibility = 'hidden'
#     Mode_w.value="---"
#################################

In [ ]:
print("Application Web de retouche d'images :")

In [ ]:
file = open("Image3b.jpg", "rb")
image = file.read()
Img = widgets.Image(
    value=image,
    format='jpg',
    width=600,
    height=550,
)

In [ ]:
####  Contraste  #####
clair_z = widgets.IntSlider(min = 0, max = 60,  value = 0 , description='+ clair')
sombre_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='+ sombre')
cl_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='cl_new')
fo_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='fo_new')
clair_z.observe(Contrast, names='value')
sombre_z.observe(Contrast, names='value')
#####################

In [ ]:
####  Dessin  #####
Cany_w = widgets.ToggleButton(
    value=False,
    description='Dessin',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour obtenir le dessin de contour",
    icon='pencil-alt' # (FontAwesome names without the `fa-` prefix)
)
Cany_w.observe(Canny_transfo,names='value')
#####################

In [ ]:
####  Invert  #####
Invert_w = widgets.ToggleButton(
    value=False,
    description='Inverse',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour inverser le noir et le blanc",
    icon='pencil-alt' # (FontAwesome names without the `fa-` prefix)
)
Invert_w.observe(invert,names='value')
#####################

In [ ]:
Contraste_Boite = HBox([clair_z,sombre_z], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
Cany_invert_Boite = HBox([Cany_w, Invert_w], layout=Layout(margin='0 0 0 100px'))

Menu_Boite1 = VBox([Contraste_Boite, Cany_invert_Boite])
Menus = HBox([Img, Menu_Boite1], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
display(Menus)

In [ ]:
camera = CameraStream(constraints={'facingMode': {"exact": "environment"},'audio': False,'video': { 'width': 640, 'height': 480}})
# camera.layout.width='20%'
image_recorder = ImageRecorder(stream=camera, tooltip='Cliquer pour tenter une Capture')

btn_stop = Button(description="Envois à l'Iris",tooltip='Cette capture est Ok ?',button_style='success', icon="eye")
btn_stop.on_click(Recup_image)

yo=HBox([camera,VBox([btn_stop,image_recorder])], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))

# yo.layout.visibility = 'hidden'
display(yo)